In [22]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetch()
    df = pd.DataFrame(res_rows)
    return df

INFO:root:connection created


In [23]:
tornado_contracts = [
'0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936',
'0x910cbd523d972eb0a6f4cae4618ad62622b39dbf',
'0xa160cdab225685da1d56aa342ad8841c3b53f291',
'0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc',
'0x07687e702b410fa43f4cb4af7fa097918ffd2730',
'0x23773e65ed146a459791799d01336db287f25334',
'0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f',
'0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d',
'0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144',
'0x169ad27a470d064dede56a2d3ff727986b15d52b',
'0xd96f2b1c14db8458374d9aca76e26c3d18364307',
'0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3',
'0xbb93e510bbcd0b7beb5a853875f9ec60275cf498',
'0x610b717796ad172b316836ac95a2ffad065ceab4',
'0x178169b423a011fff22b9e3f3abea13414ddd0f1',
'0x03893a7c7463ae47d46bc7f091665f1893656003',
'0x2717c5e28cf931547b621a5dddb772ab6a35b701',
'0xd21be7248e0197ee08e0c20d4a96debdac3d20af'
]
tornado_contract_str = ",".join('"'+c+'"' for c in tornado_contracts)
print(tornado_contract_str)

"0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936","0x910cbd523d972eb0a6f4cae4618ad62622b39dbf","0xa160cdab225685da1d56aa342ad8841c3b53f291","0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc","0x07687e702b410fa43f4cb4af7fa097918ffd2730","0x23773e65ed146a459791799d01336db287f25334","0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f","0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d","0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144","0x169ad27a470d064dede56a2d3ff727986b15d52b","0xd96f2b1c14db8458374d9aca76e26c3d18364307","0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3","0xbb93e510bbcd0b7beb5a853875f9ec60275cf498","0x610b717796ad172b316836ac95a2ffad065ceab4","0x178169b423a011fff22b9e3f3abea13414ddd0f1","0x03893a7c7463ae47d46bc7f091665f1893656003","0x2717c5e28cf931547b621a5dddb772ab6a35b701","0xd21be7248e0197ee08e0c20d4a96debdac3d20af"


In [26]:
sql = """
insert into dw.dws_account_tag
select account_address
    ,'eth' as chain
    ,'tornado_cash_sender' as reference_tag_name
    ,rank() over(order by tornado_deposit_cnt desc) as rk
    ,min_ts as first_generate_time
    ,max_ts as last_generate_time
    ,NOW() AS etl_time
from
(
    select from_address as account_address
        ,count(*) as tornado_deposit_cnt
        ,min(transfer_time) as min_ts
        ,max(transfer_time) as max_ts
    from
    (
        select transaction_hash
            ,sum(case when from_address = "0xd90e2f925da726b50c4ed8d0fb90ad053324f31b" then 1 else 0 end) as deposit_trf_cnt
            ,sum(case when from_address in ({tornado_contracts})
                       and to_address <> '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b' 
                 then 1 else 0 end) as withdraw_trf_cnt
        from dw.dwb_token_transfer_detail_eth_hi
        where to_address in ({tornado_contracts})
        or from_address in ({tornado_contracts})
        group by 1
    ) as a
    left join dw.dwb_token_transfer_detail_eth_hi as b
    on a.transaction_hash = b.transaction_hash
    and b.to_address = '0x0000000000000000000000000000000000000000'
    where a.deposit_trf_cnt > 0
    and withdraw_trf_cnt = 0
    group by 1
) as t
""".format(tornado_contracts=tornado_contract_str)
print(sql)


insert into dw.dws_account_tag
select account_address
    ,'eth' as chain
    ,'tornado_cash_sender' as reference_tag_name
    ,rank() over(order by tornado_deposit_cnt desc) as rk
    ,min_ts as first_generate_time
    ,max_ts as last_generate_time
    ,NOW() AS etl_time
from
(
    select from_address as account_address
        ,count(*) as tornado_deposit_cnt
        ,min(transfer_time) as min_ts
        ,max(transfer_time) as max_ts
    from
    (
        select transaction_hash
            ,sum(case when from_address = "0xd90e2f925da726b50c4ed8d0fb90ad053324f31b" then 1 else 0 end) as deposit_trf_cnt
            ,sum(case when from_address in ("0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936","0x910cbd523d972eb0a6f4cae4618ad62622b39dbf","0xa160cdab225685da1d56aa342ad8841c3b53f291","0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc","0x07687e702b410fa43f4cb4af7fa097918ffd2730","0x23773e65ed146a459791799d01336db287f25334","0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f","0x4736dcf1b7a3d580672cce6e7

In [27]:
cursor.execute(sql)

12920

In [34]:
min_block_sql = """
select min(block_number) as block_number
from prod.blocks
where `chain` = 'eth'
and FROM_UNIXTIME(`timestamp`)> DATE_SUB(now(),INTERVAL 1 DAY)
"""
cursor.execute(min_block_sql)
min_block = cursor.fetchone()[0]

15351780


In [29]:
import pymysql
import logging
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")

tornado_contracts = [
'0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936',
'0x910cbd523d972eb0a6f4cae4618ad62622b39dbf',
'0xa160cdab225685da1d56aa342ad8841c3b53f291',
'0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc',
'0x07687e702b410fa43f4cb4af7fa097918ffd2730',
'0x23773e65ed146a459791799d01336db287f25334',
'0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f',
'0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d',
'0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144',
'0x169ad27a470d064dede56a2d3ff727986b15d52b',
'0xd96f2b1c14db8458374d9aca76e26c3d18364307',
'0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3',
'0xbb93e510bbcd0b7beb5a853875f9ec60275cf498',
'0x610b717796ad172b316836ac95a2ffad065ceab4',
'0x178169b423a011fff22b9e3f3abea13414ddd0f1',
'0x03893a7c7463ae47d46bc7f091665f1893656003',
'0x2717c5e28cf931547b621a5dddb772ab6a35b701',
'0xd21be7248e0197ee08e0c20d4a96debdac3d20af'
]
tornado_contract_str = ",".join('"'+c+'"' for c in tornado_contracts)

min_block_sql = """
select min(block_number) as block_number
from prod.blocks
where `chain` = 'eth'
and FROM_UNIXTIME(`timestamp`)> DATE_SUB(now(),INTERVAL 1 DAY)
"""
cursor.execute(min_block_sql)
min_block = cursor.fetchone()[0]

sql = """
insert into dw.dws_account_tag
select account_address
    ,'eth' as chain
    ,'tornado_cash_sender' as reference_tag_name
    ,rank() over(order by tornado_deposit_cnt desc) as rk
    ,min_ts as first_generate_time
    ,max_ts as last_generate_time
    ,NOW() AS etl_time
from
(
    select from_address as account_address
        ,count(*) as tornado_deposit_cnt
        ,min(transfer_time) as min_ts
        ,max(transfer_time) as max_ts
    from
    (
        select transaction_hash
            ,sum(case when from_address = "0xd90e2f925da726b50c4ed8d0fb90ad053324f31b" then 1 else 0 end) as deposit_trf_cnt
            ,sum(case when from_address in ({tornado_contracts})
                       and to_address <> '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b' 
                 then 1 else 0 end) as withdraw_trf_cnt
        from dw.dwb_token_transfer_detail_eth_hi
        where 
        (
        to_address in ({tornado_contracts})
        or from_address in ({tornado_contracts})
        )
        and block_number > {min_block}
        group by 1
    ) as a
    left join dw.dwb_token_transfer_detail_eth_hi as b
    on a.transaction_hash = b.transaction_hash
    and b.to_address = '0x0000000000000000000000000000000000000000'
    and b.block_number > {min_block}
    where a.deposit_trf_cnt > 0
    and a.withdraw_trf_cnt = 0
    group by 1
) as t
""".format(
    tornado_contracts=tornado_contract_str,
    min_block=min_block
)
# print(sql)
cursor.execute(sql)

INFO:root:connection created


ProgrammingError: (1064, 'Not support use subquery in ON clause')